In [1]:
import ROOT
import array as array
from array import array 
import os
import glob
import ROOT
from ROOT import TH1D, TH2D, TTree,TFile, TUnfoldSys, TUnfoldDensity, TUnfold, TDirectoryFile
from random import gauss
import numpy as np
import root_numpy as rtnpy
from root_numpy import *
import h5py
#from awkward import JaggedArray, Table
import matplotlib.pyplot as plt
import seaborn as sns

Welcome to JupyROOT 6.18/00



### Get list of nanoskim files to run over and sum gen event weights (ie, n_events) for a given samples

In [2]:
%jsroot on

In [3]:
def getListOfFiles(dirName, histos=False):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    #To ensure event counts don't come up against  a histogram file and rather just the nanoskim
    if histos==False: ftype="nanoskim"
    else: ftype="histogram"
    listOfFile[:] = [x for x in listOfFile if "%s"%ftype in x]
    #print (listOfFile)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):            
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

#print (getListOfFiles('/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_ZZ_v1/201125_043451/0000/'))

In [4]:
dict_MCscaling_UL17 = {
    'TTJets': ["722.8", 286629789811.7677, #xsdb
               '../TTJets_amcatnloFXFX-pythia8_2017UL.root',
              ],
    'TTToSemiLeptonic': ["365.34", 32392787477.907993,
                         '../TTToSemileptonic_powheg_pythia8_2017UL.root',
                        ],
    'TTTo2L2Nu': ["88.29 ", 4759777104.469413, 
                  '../TTTo2L2Nu_powheg_pythia8_2017UL.root',
                 ],
    'WJetsToLNu': ["5.368e+04", 9949891586212.455078, 
                   '../WJetsToLNu_madgraphMLM_pythia8_2017UL.root',
                  ],
    'ST_s-channel': ["3.549e+00", 70439807.475972, 
                     '../ST_s-channel_amcatnlo_pythia8_2017UL.root',
                    ],
    'ST_t-channel_top': ["1.197e+02", 656310119.274000, 
                         '../ST_t-channel_top_powheg_pythia8_2017UL.root'
                        ],
    'ST_t-channel_antitop': ["7.174e+01", 265124234.844000,  #xsdb says 82.52
                            '../ST_t-channel_antitop_powheg_pythia8_2017UL.root'
                            ],
    'ST_tW_top': ["3.245e+01", 331239851.967700, 
                  '../ST_tW_top_powheg_pythia8_2017UL.root',
                 ],
    'ST_tW_antitop': ["3.251e+01", 303960792.435600, 
                     '../ST_tW_antitop_powheg_pythia8_2017UL.root',
                     ],
    'QCD_Pt_170to300': ["7055.0", 36027673.000000, 
                        '../QCD_Muen_Pt_170to300_pythia8_2017UL.root',
                       ],
    'QCD_Pt_300to470': ["619.3", 494796.072880, 
                        '../QCD_Muen_Pt_300to470_pythia8_2017UL.root',   
                       ],
    'QCD_Pt_470to600': ["59.24", 517383.713837, 
                        '../QCD_Muen_Pt_470to600_pythia8_2017UL.root',
                       ],
    'QCD_Pt_600to800': ["18.21", 17318812.571925, 
                        '../QCD_Muen_Pt_600to800_pythia8_2017UL.root',
                       ],
    'QCD_Pt_800to1000': ["2.597e+01", 16962615.000000, 
                         '../QCD_Muen_Pt_800to1000_pythia8_2017UL.root'    
                        ],
    'QCD_Pt_1000toInf': ["1.078", 14642553.000000, 
                          '../QCD_Muen_Pt_1000toInf_pythia8_2017UL.root',
                         ],
    'WW': ["7.577e+01", 15634116.199514, 
          '../WW_pythia8_2017UL.root'  
          ],
    'WZ': ["1.21e+00", 7889000.000000, 
           '../WZ_pythia8_2017UL.root'    
          ],
    'ZZ': ["2.748e+00", 2706000.000000, 
           '../ZZ_pythia8_2017UL.root'
          ]
    }
dict_MCscaling_UL18 = {
    'TTJets': ["722.8", 297902725633.38257,
               '../TTJets_amcatnloFXFX-pythia8_2018UL.root',
              ],
    'TTToSemiLeptonic': ["365.34", 31378075292.893986,
                         '../TTToSemileptonic_powheg_pythia8_2018UL.root',
                        ],
    'TTTo2L2Nu': ["88.29 ", 5063754431.9676, 
                  '../TTTo2L2Nu_powheg_pythia8_2018UL.root',
                 ],
    'WJetsToLNu': ["5.368e+04", 10484986870276.357422, 
                   '../WJetsToLNu_madgraphMLM_pythia8_2018UL.root',
                  ],
    'ST_s-channel': ["3.549e+00", 64014403.924644, 
                     '../ST_s-channel_amcatnlo_pythia8_2018UL.root',
                    ],
    'ST_t-channel_top': ["1.197e+02", 683747124.990000, 
                         '../ST_t-channel_top_powheg_pythia8_2018UL.root'
                        ],
    'ST_t-channel_antitop': ["7.174e+01", 271219283.652000, 
                            '../ST_t-channel_antitop_powheg_pythia8_2018UL.root'
                            ],
    'ST_tW_top': ["3.245e+01", 328321574.253000, 
                  '../ST_tW_top_powheg_pythia8_2018UL.root',
                 ],
    'ST_tW_antitop': ["3.251e+01", 296655487.557600, 
                     '../ST_tW_antitop_powheg_pythia8_2018UL.root',
                     ],
    'QCD_Pt_170to300': ["7055.0", 0.,#36027673.000000, 
                        '../QCD_Muen_Pt_170to300_pythia8_2018UL.root',
                       ],
    'QCD_Pt_300to470': ["619.3", 0.,#494796.072880, 
                        '../QCD_Muen_Pt_300to470_pythia8_2018UL.root',   
                       ],
    'QCD_Pt_470to600': ["59.24", 0.,#517383.713837, 
                        '../QCD_Muen_Pt_470to600_pythia8_2018UL.root',
                       ],
    'QCD_Pt_600to800': ["18.21", 0.,#17318812.571925, 
                        '../QCD_Muen_Pt_600to800_pythia8_2018UL.root',
                       ],
    'QCD_Pt_800to1000': ["2.597e+01", 0.,#39261600.000000, 
                         '../QCD_Pt_800to1000_pythia8_2018UL.root'    
                        ],
    'QCD_Pt_1000toInf': ["1.078", 0.,#14642553.000000, 
                          '../QCD_Muen_Pt_1000toInf_pythia8_2018UL.root',
                         ],
    
    'WW': ["7.577e+01", 7959266.140092, 
          '../WW_pythia8_2018UL.root'  
          ],
    'WZ': ["1.21e+00", 3976000.000000, 
           '../WZ_pythia8_2018UL.root'    
          ],
    'ZZ': ["2.748e+00", 2000000.000000, 
           '../ZZ_pythia8_2018UL.root'
          ]
    }


dict_MCscaling_combined = {
    'TTJets': ["722.8", 5.84532515e+11,
               '../TTJets_amcatnloFXFX-pythia8_UL17-18.root',
              ],
    'TTToSemiLeptonic': ["365.34", 6.37708628e+10,
                         '../TTToSemileptonic_powheg_pythia8_UL17-18.root',
                        ],
    'TTTo2L2Nu': ["88.29 ", 9.82353154e+09, 
                  '../TTTo2L2Nu_powheg_pythia8_UL17-18.root',
                 ],
    'WJetsToLNu': ["5.368e+04", 2.04348785e+13, 
                   '../WJetsToLNu_madgraphMLM_pythia8_UL17-18.root',
                  ],
    'ST_s-channel': ["3.549e+00", 1.34454211e+08, 
                     '../ST_s-channel_amcatnlo_pythia8_UL17-18.root',
                    ],
    'ST_t-channel_top': ["1.197e+02", 1.34005724e+09, 
                         '../ST_t-channel_top_powheg_pythia8_UL17-18.root'
                        ],
    'ST_t-channel_antitop': ["7.174e+01", 5.36343518e+08, 
                            '../ST_t-channel_antitop_powheg_pythia8_UL17-18.root'
                            ],
    'ST_tW_top': ["3.245e+01", 6.59561426e+08, 
                  '../ST_tW_top_powheg_pythia8_UL17-18.root',
                 ],
    'ST_tW_antitop': ["3.251e+01", 6.00616280e+08 , 
                     '../ST_tW_antitop_powheg_pythia8_UL17-18.root',
                     ],
    'QCD_Pt_170to300': ["7055.0", 36027673.000000, 
                        '../QCD_Muen_Pt_170to300_pythia8_2017UL.root',
                       ],
    'QCD_Pt_300to470': ["619.3", 494796.072880, 
                        '../QCD_Muen_Pt_300to470_pythia8_2017UL.root',   
                       ],
    'QCD_Pt_470to600': ["59.24", 517383.713837, 
                        '../QCD_Muen_Pt_470to600_pythia8_2017UL.root',
                       ],
    'QCD_Pt_600to800': ["18.21", 17318812.571925, 
                        '../QCD_Muen_Pt_600to800_pythia8_2017UL.root',
                       ],
    'QCD_Pt_800to1000': ["2.597e+01", 39261600.000000, 
                         '../QCD_Muen_Pt_800to1000_pythia8_2017UL.root'   
                        ],
    'QCD_Pt_1000toInf': ["1.078", 14642553.000000, 
                          '../QCD_Muen_Pt_1000toInf_pythia8_2017UL.root',
                         ],
    
    'WW': ["7.577e+01", 2.35933823e+07, 
          '../WW_pythia8_UL17-18.root'  
          ],
    'WZ': ["1.21e+00", 1.18650000e+07, 
           '../WZ_pythia8_UL17-18.root'    
          ],
    'ZZ': ["2.748e+00", 4.70600000e+06, 
           '../ZZ_pythia8_UL17-18.root'
          ]
    }


In [5]:
lumi=41472.052#4653.71+9289.53+4247.68+9312.83+11758.83
print (lumi)

samples = list(dict_MCscaling_UL17.keys())
dict_items = list(dict_MCscaling_UL17.items())
xsecs = np.array(list(float(dict_items[k][1][0]) for k in np.arange(0,len(dict_items)))) 
nevts = np.array(list(float(dict_items[k][1][1]) for k in np.arange(0,len(dict_items)))) 
fpathsMC = np.array(list((dict_items[k][1][2]) for k in np.arange(0,len(dict_items))))
fpathData = '../SingleMuonAll_2017UL.root'
for i in range(0,len(samples)):
    print (i, xsecs[i], samples[i], nevts[i], fpathsMC[i])

41472.052
0 722.8 TTJets 286629789811.7677 ../TTJets_amcatnloFXFX-pythia8_2017UL.root
1 365.34 TTToSemiLeptonic 32392787477.907993 ../TTToSemileptonic_powheg_pythia8_2017UL.root
2 88.29 TTTo2L2Nu 4759777104.469413 ../TTTo2L2Nu_powheg_pythia8_2017UL.root
3 53680.0 WJetsToLNu 9949891586212.455 ../WJetsToLNu_madgraphMLM_pythia8_2017UL.root
4 3.549 ST_s-channel 70439807.475972 ../ST_s-channel_amcatnlo_pythia8_2017UL.root
5 119.7 ST_t-channel_top 656310119.274 ../ST_t-channel_top_powheg_pythia8_2017UL.root
6 71.74 ST_t-channel_antitop 265124234.844 ../ST_t-channel_antitop_powheg_pythia8_2017UL.root
7 32.45 ST_tW_top 331239851.9677 ../ST_tW_top_powheg_pythia8_2017UL.root
8 32.51 ST_tW_antitop 303960792.4356 ../ST_tW_antitop_powheg_pythia8_2017UL.root
9 7055.0 QCD_Pt_170to300 36027673.0 ../QCD_Muen_Pt_170to300_pythia8_2017UL.root
10 619.3 QCD_Pt_300to470 494796.07288 ../QCD_Muen_Pt_300to470_pythia8_2017UL.root
11 59.24 QCD_Pt_470to600 517383.713837 ../QCD_Muen_Pt_470to600_pythia8_2017UL.root


In [6]:
def Fill1D(h0,h,bins):
    N=h0.GetNbinsX()
    content=np.zeros((len(bins)-1),dtype=float)
    errors=np.zeros((len(bins)-1),dtype=float)

    for i in range(len(bins)-1):
        content[i]=0
    for i in range(N):
        center=h0.GetBinCenter(i+1)
        index=np.digitize(np.array([center]),bins)
        if index[0] < len(bins):
            content[index[0]-1]+=h0.GetBinContent(i+1)
            errors[index[0]-1]+=h0.GetBinError(i+1)
    for i in range(len(bins)-1):
        h.SetBinContent(i+1,content[i])
        h.SetBinError(i+1,errors[i])

def Fill2D(h0,h,binx,biny):
    Nx=h0.GetNbinsX()
    Ny=h0.GetNbinsY()
    hx=h0.ProjectionX()
    hy=h0.ProjectionY()
    content=np.zeros((len(binx)-1,len(biny)-1),dtype=float)
    errors=np.zeros((len(binx)-1,len(biny)-1),dtype=float)
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            content[i][j]=0
    for i in range(Nx):
        for j in range(Ny):
            centerx=hx.GetBinCenter(i+1)
            indexx=np.digitize(np.array([centerx]),binx)
            centery=hy.GetBinCenter(j+1)
            indexy=np.digitize(np.array([centery]),biny)
            if indexx[0] < len(binx) and indexy[0] < len(biny):
                content[indexx[0]-1][indexy[0]-1]+=h0.GetBinContent(i+1,j+1)
                errors[indexx[0]-1][indexy[0]-1]+=ROOT.TMath.Power(h0.GetBinError(i+1,j+1),2)
                
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            h.SetBinContent(i+1,j+1,content[i][j])
            h.SetBinError(i+1,j+1,ROOT.TMath.Sqrt(errors[i][j]))


In [7]:
def DoUnfolding(Response,Reco):
    tunfolder = ROOT.TUnfoldDensity(Response,ROOT.TUnfold.kHistMapOutputHoriz,ROOT.TUnfold.kRegModeCurvature, ROOT.TUnfold.kEConstraintNone, ROOT.TUnfoldDensity.kDensityModeBinWidth)
    tunfolder.SetInput(Reco)
    tunfolder.DoUnfold(0.)
    return tunfolder.GetOutput("MC_unfolded")
def CrossClosure(response1,reco1,response2,reco2):
    unf11=DoUnfolding(response1,reco1)
    unf12=DoUnfolding(response2,reco1)
    unf21=DoUnfolding(response1,reco2)
    unf22=DoUnfolding(response2,reco2)
    return unf11,unf12,unf21,unf22
def SelfClosure(response1,reco2,response2,reco1):
    unf21=DoUnfolding(response1,reco2)
    unf12=DoUnfolding(response2,reco1)
    return unf21,unf12

In [8]:
def getRebinnedRescaledHist(hist_name, file_name, xrange, scale, isMC=True):
    #if not(sample_name in file_name): 
    #    return "File either not available or needs to be renamed to be compatible with this (poorly,) hard-coded piece of software..."
    
    FILE = ROOT.TFile.Open(file_name,'read')
    tree = FILE.Get('jetObservables')
    hist = tree.Get(hist_name)
    hist.SetDirectory(0)
    
    ROOT.TH1.AddDirectory(ROOT.kFALSE);
    
    bins = array('d', np.array(Bin(xrange)))
    
    newHist = ROOT.TH1F(hist.Rebin(len(bins)-1, "%s_rebin"%hist_name, bins))
    #newHist.SetStats(ROOT.kFALSE)
    if isMC==True: newHist.Scale(scale)
    
    return newHist

In [9]:
from array import array
ROOT.gStyle.SetOptStat(0)

In [10]:
varNames_WSel=np.array([
    'tau_0p5_1', 'tau_1_1', 'tau_2_1', 
    'tau_0p5_2', 'tau_1_2', 'tau_2_2',
    'tau_0p5_3', 'tau_1_3', 'tau_2_3',
    'tau_0p5_4', 'tau_1_4', 'tau_2_4',
    'tau21', 'tau32'
    ])
recoHist_WSel=np.array([
    'recoJet_tau_0p5_1_nom_WSel', 'recoJet_tau_1_1_nom_WSel', 'recoJet_tau_2_1_nom_WSel',
    'recoJet_tau_0p5_2_nom_WSel', 'recoJet_tau_1_2_nom_WSel', 'recoJet_tau_2_2_nom_WSel',
    'recoJet_tau_0p5_3_nom_WSel', 'recoJet_tau_1_3_nom_WSel', 'recoJet_tau_2_3_nom_WSel',
    'recoJet_tau_0p5_4_nom_WSel', 'recoJet_tau_1_4_nom_WSel', 'recoJet_tau_2_4_nom_WSel',
    'recoJet_tau21_nom_WSel', 'recoJet_tau32_nom_WSel'
    ])
fakerecoHist_WSel=np.array([
    'fakerecoJet_tau_0p5_1_nom_WSel', 'fakerecoJet_tau_1_1_nom_WSel', 'fakerecoJet_tau_2_1_nom_WSel',
    'fakerecoJet_tau_0p5_2_nom_WSel', 'fakerecoJet_tau_1_2_nom_WSel', 'fakerecoJet_tau_2_2_nom_WSel',
    'fakerecoJet_tau_0p5_3_nom_WSel', 'fakerecoJet_tau_1_3_nom_WSel', 'fakerecoJet_tau_2_3_nom_WSel',
    'fakerecoJet_tau_0p5_4_nom_WSel', 'fakerecoJet_tau_1_4_nom_WSel', 'fakerecoJet_tau_2_4_nom_WSel',
    'fakerecoJet_tau21_nom_WSel', 'fakerecoJet_tau32_nom_WSel',
    ])
truerecoHist_WSel=np.array([
    'truerecoJet_tau_0p5_1_nom_WSel', 'truerecoJet_tau_1_1_nom_WSel', 'truerecoJet_tau_2_1_nom_WSel',
    'truerecoJet_tau_0p5_2_nom_WSel', 'truerecoJet_tau_1_2_nom_WSel', 'truerecoJet_tau_2_2_nom_WSel',
    'truerecoJet_tau_0p5_3_nom_WSel', 'truerecoJet_tau_1_3_nom_WSel', 'truerecoJet_tau_2_3_nom_WSel',
    'truerecoJet_tau_0p5_4_nom_WSel', 'truerecoJet_tau_1_4_nom_WSel', 'truerecoJet_tau_2_4_nom_WSel',
    'truerecoJet_tau21_nom_WSel', 'truerecoJet_tau32_nom_WSel',
    ])
genHist_WSel=np.array([
    'genJet_tau_0p5_1_WSel', 'genJet_tau_1_1_WSel', 'genJet_tau_2_1_WSel',
    'genJet_tau_0p5_2_WSel', 'genJet_tau_1_2_WSel', 'genJet_tau_2_2_WSel',
    'genJet_tau_0p5_3_WSel', 'genJet_tau_1_3_WSel', 'genJet_tau_2_3_WSel',
    'genJet_tau_0p5_4_WSel', 'genJet_tau_1_4_WSel', 'genJet_tau_2_4_WSel',
    'genJet_tau21_WSel', 'genJet_tau32_WSel',
    ])
accepgenHist_WSel=np.array([
    'accepgenJet_tau_0p5_1_WSel', 'accepgenJet_tau_1_1_WSel', 'accepgenJet_tau_2_1_WSel',
    'accepgenJet_tau_0p5_2_WSel', 'accepgenJet_tau_1_2_WSel', 'accepgenJet_tau_2_2_WSel',
    'accepgenJet_tau_0p5_3_WSel', 'accepgenJet_tau_1_3_WSel', 'accepgenJet_tau_2_3_WSel',
    'accepgenJet_tau_0p5_4_WSel', 'accepgenJet_tau_1_4_WSel', 'accepgenJet_tau_2_4_WSel',
    'accepgenJet_tau21_WSel', 'accepgenJet_tau32_WSel',
    ])
missgenHist_WSel=np.array([
    'missgenJet_tau_0p5_1_WSel', 'missgenJet_tau_1_1_WSel', 'missgenJet_tau_2_1_WSel',
    'missgenJet_tau_0p5_2_WSel', 'missgenJet_tau_1_2_WSel', 'missgenJet_tau_2_2_WSel',
    'missgenJet_tau_0p5_3_WSel', 'missgenJet_tau_1_3_WSel', 'missgenJet_tau_2_3_WSel',
    'missgenJet_tau_0p5_4_WSel', 'missgenJet_tau_1_4_WSel', 'missgenJet_tau_2_4_WSel',
    'missgenJet_tau21_WSel', 'missgenJet_tau32_WSel',
    ])

response_WSel=np.array([
    'respJet_tau_0p5_1_nom_WSel', 'respJet_tau_1_1_nom_WSel', 'respJet_tau_2_1_nom_WSel',
    'respJet_tau_0p5_2_nom_WSel', 'respJet_tau_1_2_nom_WSel', 'respJet_tau_2_2_nom_WSel',
    'respJet_tau_0p5_3_nom_WSel', 'respJet_tau_1_3_nom_WSel', 'respJet_tau_2_3_nom_WSel',
    'respJet_tau_0p5_4_nom_WSel', 'respJet_tau_1_4_nom_WSel', 'respJet_tau_2_4_nom_WSel',
    'respJet_tau21_nom_WSel', 'respJet_tau32_nom_WSel',
    ])
HistLabels_WSel=np.array([
    ['Events/0.03','#tau_{1}^{(0.5)}','#tau_{1}^{(0.5)}'],['Events/0.02','#tau_{1}^{(1)}','#tau_{1}^{(1)}'],['Events/0.02','#tau_{1}^{(2)}','#tau_{1}^{(2)}'],
    ['Events/0.03','#tau_{2}^{(0.5)}','#tau_{2}^{(0.5)}'],['Events/0.02','#tau_{2}^{(1)}','#tau_{2}^{(1)}'],['Events/0.01','#tau_{2}^{(2)}','#tau_{2}^{(2)}'],
    ['Events/0.02','#tau_{3}^{(0.5)}','#tau_{3}^{(0.5)}'],['Events/0.02','#tau_{3}^{(1)}','#tau_{3}^{(1)}'],['Events/0.01','#tau_{3}^{(2)}','#tau_{3}^{(2)}'],
    ['Events/0.02','#tau_{4}^{(0.5)}','#tau_{4}^{(0.5)}'],['Events/0.01','#tau_{4}^{(1)}','#tau_{4}^{(1)}'],['Events/0.01','#tau_{4}^{(2)}','#tau_{4}^{(2)}'],
    ['Events/0.01','#tau_{2,1}^{(1)}','#tau_{2,1}^{(1)}'],['Events/0.01','#tau_{3,2}^{(1)}','#tau_{3,2}^{(1)}']
    ])
genBin_WSel=[
    array('d',[0., 0.2, 0.26, 0.32, 0.38, 0.44,  0.5, 0.56, 0.62, 0.7, 0.78, 0.9]),
    array('d',[0., 0.1, 0.18, 0.24, 0.3, 0.36, 0.4, 0.46, 0.52, 0.6]), 
    array('d',[0., 0.04, 0.08, 0.12, 0.16, 0.2, 0.26, 0.32, 0.4, 0.56]),
    array('d',[0., 0.12, 0.18, 0.24, 0.3, 0.36, 0.42,  0.48,  0.54, 0.66]),
    array('d',[0., 0.04, 0.08, 0.12, 0.16, 0.2, 0.24, 0.28, 0.32, 0.4]), 
    array('d',[0., 0.01, 0.02, 0.03, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.2]), 
    array('d',[0., 0.1, 0.14, 0.2, 0.26, 0.32, 0.38, 0.44, 0.5, 0.56]), 
    array('d',[0., 0.02, 0.04, 0.06, 0.08, 0.1, 0.13, 0.16, 0.2, 0.25]), 
    array('d',[0., 0.01, 0.02, 0.03, 0.04, 0.06, 0.1, 0.16]), 
    array('d',[0., 0.1, 0.14, 0.18, 0.22, 0.26, 0.3, 0.34, 0.4, 0.46]), 
    array('d',[0., 0.02, 0.04, 0.06, 0.08, 0.1, 0.13, 0.16, 0.2, 0.25]), 
    array('d',[0., 0.01, 0.02, 0.03, 0.04, 0.05, 0.064]),
    array('d',[0., 0.1, 0.18, 0.26, 0.34, 0.42, 0.5, 0.58, 0.66, 0.74, 0.87, 1.]),
     
    ]
recoBin_WSel = np.copy(genBin_WSel)

for i in range(len(genBin_WSel)):
    gen_bin = genBin_WSel[i]
    reco_bin = array('d', np.sort(np.append(gen_bin, np.array([np.round((gen_bin[j]+gen_bin[j+1])/2,3) for j in range(len(gen_bin)-1)]))))
    #print (len(gen_bin), gen_bin, "\n", len(reco_bin), reco_bin)
    recoBin_WSel[i] = reco_bin
    
for i in range(len(genBin_WSel)):
    print (varNames_WSel[i], genBin_WSel[i], recoBin_WSel[i])


tau_0p5_1 array('d', [0.0, 0.2, 0.26, 0.32, 0.38, 0.44, 0.5, 0.56, 0.62, 0.7, 0.78, 0.9]) array('d', [0.0, 0.1, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.41, 0.44, 0.47, 0.5, 0.53, 0.56, 0.59, 0.62, 0.66, 0.7, 0.74, 0.78, 0.84, 0.9])
tau_1_1 array('d', [0.0, 0.1, 0.18, 0.24, 0.3, 0.36, 0.4, 0.46, 0.52, 0.6]) array('d', [0.0, 0.05, 0.1, 0.14, 0.18, 0.21, 0.24, 0.27, 0.3, 0.33, 0.36, 0.38, 0.4, 0.43, 0.46, 0.49, 0.52, 0.56, 0.6])
tau_2_1 array('d', [0.0, 0.04, 0.08, 0.12, 0.16, 0.2, 0.26, 0.32, 0.4, 0.56]) array('d', [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.23, 0.26, 0.29, 0.32, 0.36, 0.4, 0.48, 0.56])
tau_0p5_2 array('d', [0.0, 0.12, 0.18, 0.24, 0.3, 0.36, 0.42, 0.48, 0.54, 0.66]) array('d', [0.0, 0.06, 0.12, 0.15, 0.18, 0.21, 0.24, 0.27, 0.3, 0.33, 0.36, 0.39, 0.42, 0.45, 0.48, 0.51, 0.54, 0.6, 0.66])
tau_1_2 array('d', [0.0, 0.04, 0.08, 0.12, 0.16, 0.2, 0.24, 0.28, 0.32, 0.4]) array('d', [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.

# All $\tau_N$ self closure 80:20 split Powheg+Pythia8 (nominal)

In [11]:
def Scale(h):
    n=h.Integral()
    h.Scale(1./n,"width")

In [12]:
scale1=lumi*xsecs[0]/24408467409.655979 #big split
scale2=lumi*xsecs[0]/7984320068.252006 #small split

In [13]:
fData=[]
path='../SingleMuonAll_2017UL.root'
files=glob.glob(path)
for f in files:
    fData.append(f)
print (len(fData))

fMC=[
    '../TTToSemileptonic_UL17_big.root',
    '../TTToSemileptonic_UL17_small.root'
]
print (len(fMC))

1
2


In [14]:
print (scale1,scale2)

0.0012280983759652812 0.003754358408650644


In [15]:
canvases2 = []
for i in range(len(genBin_WSel)):
    c = ROOT.TCanvas("c%d"%i,"c%d"%i,10, 10, 750, 750 )
    canvases2.append(c)
    

for i in range(len(canvases2)):
    
    
    print (varNames_WSel[i], genBin_WSel[i], recoBin_WSel[i])
    gen_bin = genBin_WSel[i]
    reco_bin = recoBin_WSel[i]
    
    can=canvases2[i]
    
    MC1_reco=ROOT.TH1D("MC_reco Powheg+P8_75pc","MC_reco Powheg+P8_75pc;#tau_{1}^{(0.5)};Events/0.01",len(reco_bin)-1,reco_bin)
    MC1_gen=ROOT.TH1D("MC_gen Powheg+P8_75pc","MC_gen Powheg+P8_75pc;#tau_{1}^{(0.5)};Events/0.01",len(gen_bin)-1,gen_bin)
    Response1=ROOT.TH2D("Response matrix Powheg+P8_75pc","Response matrix Powheg+P8_75pc;#tau_{1}^{(0.5)} gen;#tau_{1}^{(0.5)} reco",len(gen_bin)-1,gen_bin,len(reco_bin)-1,reco_bin)
    MC2_reco=ROOT.TH1D("MC_reco Powheg+P8_25pc","MC_reco Powheg+P8_25pc;#tau_{1}^{(0.5)};Events/0.01",len(reco_bin)-1,reco_bin)
    MC2_gen=ROOT.TH1D("MC_gen Powheg+P8_25pc","MC_gen Powheg+P8_25pc;#tau_{1}^{(0.5)};Events/0.01",len(gen_bin)-1,gen_bin)
    Response2=ROOT.TH2D("Response matrix Powheg+P8_25pc","Response matrix Powheg+P8_25pc;#tau_{1}^{(0.5)} gen;#tau_{1}^{(0.5)} reco",len(gen_bin)-1,gen_bin,len(reco_bin)-1,reco_bin)
    
    print ("loading files")
    file_MC=np.zeros((len(fMC),),dtype=ROOT.TFile)
    file_MC[0]=ROOT.TFile.Open(fMC[0],'read')
    t=file_MC[0].Get('jetObservables')
    MC1_reco0 = t.Get(recoHist_WSel[i])
    #print (MC1_reco0.GetNbinsX())
    MC1_reco0.Scale(scale1)
    MC1_reco0.SetDirectory(0)
    MC1_gen0 = t.Get(accepgenHist_WSel[i])
    MC1_gen0.Scale(scale1)
    MC1_gen0.SetDirectory(0)
    Response1_0 = t.Get(response_WSel[i])
    Response1_0.Scale(scale1)
    Response1_0.SetDirectory(0)
    file_MC[0].Close()
    

    file_MC[1]=ROOT.TFile.Open(fMC[1],'read')
    t=file_MC[1].Get('jetObservables')
    MC2_reco0 = t.Get(recoHist_WSel[i])
    MC2_reco0.Scale(scale2)
    MC2_reco0.SetDirectory(0)
    MC2_gen0  = t.Get(accepgenHist_WSel[i])
    MC2_gen0.Scale(scale2)
    MC2_gen0.SetDirectory(0)
    Response2_0 = t.Get(response_WSel[i])
    Response2_0.Scale(scale2)
    Response2_0.SetDirectory(0)
    file_MC[1].Close()   
    
    
    
    print ("Re-filling histos")
    #print ("Filling resp")
    Fill1D(MC1_reco0,MC1_reco,reco_bin)
    Fill1D(MC1_gen0,MC1_gen,gen_bin)
    Fill2D(Response1_0,Response1,gen_bin,reco_bin)
    Fill1D(MC2_reco0,MC2_reco,reco_bin)
    Fill1D(MC2_gen0,MC2_gen,gen_bin)
    Fill2D(Response2_0,Response2,gen_bin,reco_bin)    #print ("Filling MC2 reco")
    Fill1D(MC2_reco0,MC2_reco,reco_bin)
    #print ("Filling MC2 gen")
    Fill1D(MC2_gen0,MC2_gen,gen_bin)
    
    can.cd()
    pad1 = ROOT.TPad("pad1", "Main",0,0.207,1.00,1.00,-1)
    pad2 = ROOT.TPad("pad2", "Ratio",0,0.00,1.00,0.30,-1)
    pad1.Draw()
    pad2.Draw()

    pad1.cd()
    
    if not(varNames_WSel[i] in ["tau_0p5_1", "tau_1_1", "tau32"]): legend1=ROOT.TLegend(0.65,0.65,0.90,0.88)
    else: legend1=ROOT.TLegend(0.15,0.65,0.35,0.88)
    
    legend1.SetFillStyle(0)
    legend1.SetBorderSize(0)

    print ("Unfolding and scaling histos")
    
    unf21,unf12=SelfClosure(Response1,MC2_reco,Response2,MC1_reco)
    #Scale(MC1_gen)
    Scale(unf21)
    Scale(MC2_gen)
    #Scale(unf12)
    
    unf21.SetLineColor(ROOT.kPink-1)
    unf21.SetMarkerColor(ROOT.kPink-1)
    unf21.SetMarkerStyle(26)
    unf21.SetMarkerSize(1)
    unf21.SetLineWidth(2)
    legend1.AddEntry(unf21,"Unfolded","lep")
    unf21.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(%s)}'%HistLabels_WSel[i][1] )
    unf21.Draw("e0")

    MC2_gen.SetLineColor(ROOT.kCyan-6)
    MC2_gen.SetLineWidth(2)
    MC2_gen.SetMarkerSize(2)
    legend1.AddEntry(MC2_gen,"Accepted Gen","l")
    MC2_gen.Draw("e0same")
    
    legend1.Draw("same")
    can.Update()
    
    pad2.cd()
    ROOT.gStyle.SetOptFit(1)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.3)

    legend2=ROOT.TLegend(0.1,0.3,0.6,0.65)
    legend2.SetBorderSize(0)
    legend2.SetFillStyle(0)
    
    ratio1=unf21.Clone("ratio1")
    ratio1.Divide(MC2_gen)
    ratio1.SetTitle("")
    ratio1.GetXaxis().SetTitle(HistLabels_WSel[i][1])
    ratio1.SetTitleSize(0)
    ratio1.GetXaxis().SetTitleSize(0.12)
    ratio1.GetXaxis().SetLabelSize(0.12)
    ratio1.GetYaxis().SetTitle("(Unf.)/(Accep. gen)")
    ratio1.GetYaxis().SetTitleSize(0.11)
    ratio1.GetYaxis().SetTitleOffset(0.42)
    ratio1.GetYaxis().SetLabelSize(0.12)
    ratio1.GetYaxis().SetRangeUser(0.4,1.6)
    ratio1.SetMarkerStyle(22)
    ratio1.SetMarkerColor(ROOT.kPink-1)
    ratio1.SetLineColor(ROOT.kPink-1)
    ratio1.Draw("P0")
    #legend2.AddEntry(ratio1,"(Unfolding)/(Accepted gen)","lep")
    
    legend2.Draw("Same")
    can.Update()
    can.Draw()
    can.Print("Plots/SelfClosureWSel_%s_7525_UL17.png"%varNames_WSel[i] )
    MC2_gen.Delete()
    MC2_reco.Delete()
    Response1.Delete()
    can.Delete()

tau_0p5_1 array('d', [0.0, 0.2, 0.26, 0.32, 0.38, 0.44, 0.5, 0.56, 0.62, 0.7, 0.78, 0.9]) array('d', [0.0, 0.1, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.41, 0.44, 0.47, 0.5, 0.53, 0.56, 0.59, 0.62, 0.66, 0.7, 0.74, 0.78, 0.84, 0.9])
loading files
Re-filling histos
Unfolding and scaling histos
tau_1_1 array('d', [0.0, 0.1, 0.18, 0.24, 0.3, 0.36, 0.4, 0.46, 0.52, 0.6]) array('d', [0.0, 0.05, 0.1, 0.14, 0.18, 0.21, 0.24, 0.27, 0.3, 0.33, 0.36, 0.38, 0.4, 0.43, 0.46, 0.49, 0.52, 0.56, 0.6])
loading files
Re-filling histos
Unfolding and scaling histos
tau_2_1 array('d', [0.0, 0.04, 0.08, 0.12, 0.16, 0.2, 0.26, 0.32, 0.4, 0.56]) array('d', [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.23, 0.26, 0.29, 0.32, 0.36, 0.4, 0.48, 0.56])
loading files
Re-filling histos
Unfolding and scaling histos
tau_0p5_2 array('d', [0.0, 0.12, 0.18, 0.24, 0.3, 0.36, 0.42, 0.48, 0.54, 0.66]) array('d', [0.0, 0.06, 0.12, 0.15, 0.18, 0.21, 0.24, 0.27, 0.3, 0.33, 0.36, 0.39, 0.42, 0.45, 0.48, 0.

Info in <TUnfold::SetConstraint>: fConstraint=0
Warning in <TUnfold::TUnfold>: 3 output bins do not depend on the input data  0 11-12
Info in <TUnfold::TUnfold>: 22 input bins and 10 output bins
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #0 (xaxis:#tau_{1}^{(0.5)} gen[ufl])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #11 (xaxis:#tau_{1}^{(0.5)} gen[0.78,0.9])
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #12 (xaxis:#tau_{1}^{(0.5)} gen[ofl])
Info in <TUnfoldDensity::RegularizeOneDistribution>: regularizing xaxis regMode=3 densityMode=1 axisSteering=*[UOB]
Warning in <TUnfoldDensity::SetInput>: 3/22 input bins have zero error, and are ignored.
Info in <TUnfold::SetConstraint>: fConstraint=0
Warning in <TUnfold::TUnfold>: 3 output bins do not depend on the input data  0 11-12
Info in <TUnfold::TUnfold>: 22 input bins and 10 output bins
Info in <TUnfoldDensity::TUnfold>: *NOT* unfolding bin #0 (xaxis:#tau_{1}^{(0.5)} gen[ufl])
Info in <TUnfoldDensity::TUnfo

## UL17: All $\tau_N$ cross-closure Powheg+Pythia8 (nominal) vs. aMC@NLOFXFX+Pythia8

In [16]:
scale1=lumi*xsecs[0]/nevts[0] #powheg+pythia8
scale2=lumi*xsecs[1]/nevts[1] #aMC@NLOFXFX+pythia8

fData=[]
path='../SingleMuonAll_2017UL.root'
files=glob.glob(path)
for f in files:
    fData.append(f)
print (len(fData))

fMC=[
    '../TTToSemileptonic_powheg_pythia8_2017UL.root',
    '../TTJets_amcatnloFXFX-pythia8_2017UL.root'
]

print (len(fMC))

1
2


In [17]:
canvases2 = []
for i in range(len(genBin_WSel)):
    c = ROOT.TCanvas("c%d"%i,"c%d"%i,10, 10, 750, 750 )
    canvases2.append(c)
    

for i in range(len(canvases2)):
    
    
    print (varNames_WSel[i], genBin_WSel[i], recoBin_WSel[i])
    gen_bin = genBin_WSel[i]
    reco_bin = recoBin_WSel[i]
    
    can=canvases2[i]
    
    MC1_reco=ROOT.TH1D("MC_reco Powheg+P8","MC_reco Powheg+P8;#tau_{1}^{(0.5)};Events/0.01",len(reco_bin)-1,reco_bin)
    MC1_gen=ROOT.TH1D("MC_gen Powheg+P8","MC_gen Powheg+P8;#tau_{1}^{(0.5)};Events/0.01",len(gen_bin)-1,gen_bin)
    Response1=ROOT.TH2D("Response matrix Powheg+P8","Response matrix Powheg+P8;#tau_{1}^{(0.5)} gen;#tau_{1}^{(0.5)} reco",len(gen_bin)-1,gen_bin,len(reco_bin)-1,reco_bin)
    MC2_reco=ROOT.TH1D("MC_reco aMC@NLO+Pythia8","MC_reco aMC@NLO+Pythia8;#tau_{1}^{(0.5)};Events/0.01",len(reco_bin)-1,reco_bin)
    MC2_gen=ROOT.TH1D("MC_gen aMC@NLO+Pythia8","MC_gen aMC@NLO+Pythia8;#tau_{1}^{(0.5)};Events/0.01",len(gen_bin)-1,gen_bin)
    Response2=ROOT.TH2D("Response matrix aMC@NLO+Pythia8","Response matrix aMC@NLO+Pythia8;#tau_{1}^{(0.5)} gen;#tau_{1}^{(0.5)} reco",len(gen_bin)-1,gen_bin,len(reco_bin)-1,reco_bin)
    
    print ("loading files")
    file_MC=np.zeros((len(fMC),),dtype=ROOT.TFile)
    file_MC[0]=ROOT.TFile.Open(fMC[0],'read')
    t=file_MC[0].Get('jetObservables')
    MC1_reco0 = t.Get(recoHist_WSel[i])
    #print (MC1_reco0.GetNbinsX())
    MC1_reco0.Scale(scale1)
    MC1_reco0.SetDirectory(0)
    MC1_gen0 = t.Get(accepgenHist_WSel[i])
    MC1_gen0.Scale(scale1)
    MC1_gen0.SetDirectory(0)
    Response1_0 = t.Get(response_WSel[i])
    Response1_0.Scale(scale1)
    Response1_0.SetDirectory(0)
    file_MC[0].Close()
    

    file_MC[1]=ROOT.TFile.Open(fMC[1],'read')
    t=file_MC[1].Get('jetObservables')
    MC2_reco0 = t.Get(recoHist_WSel[i])
    MC2_reco0.Scale(scale2)
    MC2_reco0.SetDirectory(0)
    MC2_gen0  = t.Get(accepgenHist_WSel[i])
    MC2_gen0.Scale(scale2)
    MC2_gen0.SetDirectory(0)
    Response2_0 = t.Get(response_WSel[i])
    Response2_0.Scale(scale2)
    Response2_0.SetDirectory(0)
    file_MC[1].Close()   
    
    
    
    print ("Re-filling histos")
    #print ("Filling resp")
    Fill1D(MC1_reco0,MC1_reco,reco_bin)
    Fill1D(MC1_gen0,MC1_gen,gen_bin)
    Fill2D(Response1_0,Response1,gen_bin,reco_bin)
    Fill1D(MC2_reco0,MC2_reco,reco_bin)
    Fill1D(MC2_gen0,MC2_gen,gen_bin)
    Fill2D(Response2_0,Response2,gen_bin,reco_bin)    #print ("Filling MC2 reco")
    Fill1D(MC2_reco0,MC2_reco,reco_bin)
    #print ("Filling MC2 gen")
    Fill1D(MC2_gen0,MC2_gen,gen_bin)
    
    can.cd()
    pad1 = ROOT.TPad("pad1", "Main",0,0.207,1.00,1.00,-1)
    pad2 = ROOT.TPad("pad2", "Ratio",0,0.00,1.00,0.30,-1)
    pad1.Draw()
    pad2.Draw()

    pad1.cd()

    if not(varNames_WSel[i] in ["tau_0p5_1", "tau_1_1", "tau32"]): legend1=ROOT.TLegend(0.65,0.65,0.90,0.88)
    else: legend1=ROOT.TLegend(0.15,0.65,0.35,0.88)
    legend1.SetFillStyle(0)
    legend1.SetBorderSize(0)

    print ("Unfolding and scaling histos")
    
    unf21,unf12=SelfClosure(Response1,MC2_reco,Response2,MC1_reco)
    Scale(MC1_gen)
    Scale(unf21)
    Scale(MC2_gen)
    Scale(unf12)
    
    unf21.SetLineColor(ROOT.kRed-7)
    unf21.SetMarkerColor(ROOT.kRed-7)
    unf21.SetMarkerStyle(26)
    unf21.SetMarkerSize(2)
    unf21.SetLineWidth(2)
    unf21.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(%s)}'%HistLabels_WSel[i][1] )
    legend1.AddEntry(unf21,"aMC@NLO unfolded","lep")
    unf21.Draw("e0")

    MC2_gen.SetLineColor(ROOT.kBlue+7)
    MC2_gen.SetLineWidth(2)
    MC2_gen.SetMarkerSize(2)
    legend1.AddEntry(MC2_gen,"Gen aMC@NLO","l")
    MC2_gen.Draw("e0same")
    
    unf12.SetLineColor(ROOT.kGreen+3)
    unf12.SetMarkerColor(ROOT.kGreen+3)
    unf12.SetMarkerStyle(26)
    unf12.SetMarkerSize(2)
    unf12.SetLineWidth(2)
    legend1.AddEntry(unf12,"Powheg unfolded","lep")
    unf12.Draw("e0same")

    MC1_gen.SetLineColor(ROOT.kGray+2)
    MC1_gen.SetLineWidth(2)
    MC1_gen.SetMarkerSize(2)
    legend1.AddEntry(MC1_gen,"Gen Powheg","l")
    MC1_gen.Draw("e0same")
    
    legend1.Draw("same")
    can.Update()
    
    pad2.cd()
    ROOT.gStyle.SetOptFit(1)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.3)

    legend2=ROOT.TLegend(0.1,0.6,0.4,0.8)
    legend2.SetBorderSize(0)
    legend2.SetFillStyle(0)
    
    ratio1=unf21.Clone("ratio1")
    ratio1.Divide(MC2_gen)
    ratio1.SetTitle("")
    ratio1.GetXaxis().SetTitle(HistLabels_WSel[i][1])
    ratio1.SetTitleSize(0)
    ratio1.GetXaxis().SetTitleSize(0.12)
    ratio1.GetXaxis().SetLabelSize(0.12)
    ratio1.GetYaxis().SetTitle("(Unf.)/(Accep. gen)")
    ratio1.GetYaxis().SetTitleSize(0.11)
    ratio1.GetYaxis().SetTitleOffset(0.42)
    ratio1.GetYaxis().SetLabelSize(0.12)
    ratio1.GetYaxis().SetRangeUser(0.4,1.6)
    ratio1.SetMarkerStyle(22)
    ratio1.SetMarkerColor(ROOT.kGray+2)
    ratio1.SetLineColor(ROOT.kGray+2)
    ratio1.Draw("P0")
    legend2.AddEntry(ratio1,"Powheg+Pythia8","lep")
    
    ratio2=unf12.Clone("ratio1")
    ratio2.Divide(MC1_gen)
    ratio2.SetTitle("")
    ratio2.GetXaxis().SetTitle(HistLabels_WSel[i][1])
    ratio2.SetTitleSize(0)
    ratio2.GetXaxis().SetTitleSize(0.12)
    ratio2.GetXaxis().SetLabelSize(0.12)
    ratio2.GetYaxis().SetTitle("(Unf.)/(Accep. gen)")
    ratio2.GetYaxis().SetTitleSize(0.11)
    ratio2.GetYaxis().SetTitleOffset(0.42)
    ratio2.GetYaxis().SetLabelSize(0.12)
    ratio2.GetYaxis().SetRangeUser(0.4,1.6)
    ratio2.SetMarkerStyle(22)
    ratio2.SetMarkerColor(ROOT.kBlue+7)
    ratio2.SetLineColor(ROOT.kBlue+7)
    ratio2.Draw("P0 same")
    legend2.AddEntry(ratio2,"aMC@NLO+Pythia8","lep")
    
    legend2.Draw("Same")
    can.Update()
    can.Draw()
    can.Print("Plots/CrossClosureWSel_%s_UL17.png"%varNames_WSel[i] )
    MC2_gen.Delete()
    MC2_reco.Delete()
    Response1.Delete()
    MC1_gen.Delete()
    MC1_reco.Delete()
    Response2.Delete()
    can.Delete()

tau_0p5_1 array('d', [0.0, 0.2, 0.26, 0.32, 0.38, 0.44, 0.5, 0.56, 0.62, 0.7, 0.78, 0.9]) array('d', [0.0, 0.1, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.41, 0.44, 0.47, 0.5, 0.53, 0.56, 0.59, 0.62, 0.66, 0.7, 0.74, 0.78, 0.84, 0.9])
loading files
Re-filling histos
Unfolding and scaling histos
tau_1_1 array('d', [0.0, 0.1, 0.18, 0.24, 0.3, 0.36, 0.4, 0.46, 0.52, 0.6]) array('d', [0.0, 0.05, 0.1, 0.14, 0.18, 0.21, 0.24, 0.27, 0.3, 0.33, 0.36, 0.38, 0.4, 0.43, 0.46, 0.49, 0.52, 0.56, 0.6])
loading files
Re-filling histos
Unfolding and scaling histos
tau_2_1 array('d', [0.0, 0.04, 0.08, 0.12, 0.16, 0.2, 0.26, 0.32, 0.4, 0.56]) array('d', [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.23, 0.26, 0.29, 0.32, 0.36, 0.4, 0.48, 0.56])
loading files
Re-filling histos
Unfolding and scaling histos
tau_0p5_2 array('d', [0.0, 0.12, 0.18, 0.24, 0.3, 0.36, 0.42, 0.48, 0.54, 0.66]) array('d', [0.0, 0.06, 0.12, 0.15, 0.18, 0.21, 0.24, 0.27, 0.3, 0.33, 0.36, 0.39, 0.42, 0.45, 0.48, 0.

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c0
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c4
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c5
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c6
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c7
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c8
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c9
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c10
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c11
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c12
Info in <TUnfold::SetConstraint>: fConstraint=0
Warning in <TUnfold::TUnfold>: 3 output

## UL18: All $\tau_N$ cross-closure Powheg+Pythia8 (nominal) vs. aMC@NLOFXFX+Pythia8

In [18]:
dict_MCscaling_UL18 = {
    'TTJets': ["722.8", 297902725633.38257,
               '../TTJets_amcatnloFXFX-pythia8_2018UL.root',
              ],
    'TTToSemiLeptonic': ["365.34", 31378075292.893986,
                         '../TTToSemileptonic_powheg_pythia8_2018UL.root',
                        ],
    'TTTo2L2Nu': ["88.29 ", 5063754431.9676, 
                  '../TTTo2L2Nu_powheg_pythia8_2018UL.root',
                 ],
    'WJetsToLNu': ["5.368e+04", 10484986870276.357422, 
                   '../WJetsToLNu_madgraphMLM_pythia8_2018UL.root',
                  ],
    'ST_s-channel': ["3.549e+00", 64014403.924644, 
                     '../ST_s-channel_amcatnlo_pythia8_2018UL.root',
                    ],
    'ST_t-channel_top': ["1.197e+02", 683747124.990000, 
                         '../ST_t-channel_top_powheg_pythia8_2018UL.root'
                        ],
    'ST_t-channel_antitop': ["7.174e+01", 271219283.652000, 
                            '../ST_t-channel_antitop_powheg_pythia8_2018UL.root'
                            ],
    'ST_tW_top': ["3.245e+01", 328321574.253000, 
                  '../ST_tW_top_powheg_pythia8_2018UL.root',
                 ],
    'ST_tW_antitop': ["3.251e+01", 296655487.557600, 
                     '../ST_tW_antitop_powheg_pythia8_2018UL.root',
                     ],
    #'QCD_Pt_170to300': ["7055.0", 36027673.000000, 
    #                    '../QCD_Muen_Pt_170to300_pythia8_2018UL.root',
    #                   ],
    #'QCD_Pt_300to470': ["619.3", 494796.072880, 
    #                    '../QCD_Muen_Pt_300to470_pythia8_2018UL.root',   
    #                   ],
    #'QCD_Pt_470to600': ["59.24", 517383.713837, 
    #                    '../QCD_Muen_Pt_470to600_pythia8_2018UL.root',
    #                   ],
    #'QCD_Pt_600to800': ["18.21", 17318812.571925, 
    #                    '../QCD_Muen_Pt_600to800_pythia8_2018UL.root',
    #                   ],
    #'QCD_Pt_800to1000': ["2.597e+01", 39261600.000000, 
    #                     '../QCD_Pt_800to1000_pythia8_2018UL.root'    
    #                    ],
    #'QCD_Pt_1000toInf': ["1.078", 14642553.000000, 
    #                      '../QCD_Muen_Pt_1000toInf_pythia8_2018UL.root',
    #                     ],
    
    'WW': ["7.577e+01", 7959266.140092, 
          '../WW_pythia8_2018UL.root'  
          ],
    'WZ': ["1.21e+00", 3976000.000000, 
           '../WZ_pythia8_2018UL.root'    
          ],
    'ZZ': ["2.748e+00", 2000000.000000, 
           '../ZZ_pythia8_2018UL.root'
          ]
    }



In [19]:
lumi=59816.39#41472.052#4653.71+9289.53+4247.68+9312.83+11758.83
print (lumi)

samples = list(dict_MCscaling_UL18.keys())
dict_items = list(dict_MCscaling_UL18.items())
xsecs = np.array(list(float(dict_items[k][1][0]) for k in np.arange(0,len(dict_items)))) 
nevts = np.array(list(float(dict_items[k][1][1]) for k in np.arange(0,len(dict_items)))) 
fpathsMC = np.array(list((dict_items[k][1][2]) for k in np.arange(0,len(dict_items))))
fpathData = '../SingleMuonAll_2018UL.root'
for i in range(0,len(samples)):
    print (i, xsecs[i], samples[i], nevts[i], fpathsMC[i])

59816.39
0 722.8 TTJets 297902725633.38257 ../TTJets_amcatnloFXFX-pythia8_2018UL.root
1 365.34 TTToSemiLeptonic 31378075292.893986 ../TTToSemileptonic_powheg_pythia8_2018UL.root
2 88.29 TTTo2L2Nu 5063754431.9676 ../TTTo2L2Nu_powheg_pythia8_2018UL.root
3 53680.0 WJetsToLNu 10484986870276.357 ../WJetsToLNu_madgraphMLM_pythia8_2018UL.root
4 3.549 ST_s-channel 64014403.924644 ../ST_s-channel_amcatnlo_pythia8_2018UL.root
5 119.7 ST_t-channel_top 683747124.99 ../ST_t-channel_top_powheg_pythia8_2018UL.root
6 71.74 ST_t-channel_antitop 271219283.652 ../ST_t-channel_antitop_powheg_pythia8_2018UL.root
7 32.45 ST_tW_top 328321574.253 ../ST_tW_top_powheg_pythia8_2018UL.root
8 32.51 ST_tW_antitop 296655487.5576 ../ST_tW_antitop_powheg_pythia8_2018UL.root
9 75.77 WW 7959266.140092 ../WW_pythia8_2018UL.root
10 1.21 WZ 3976000.0 ../WZ_pythia8_2018UL.root
11 2.748 ZZ 2000000.0 ../ZZ_pythia8_2018UL.root


In [20]:
scale1=lumi*xsecs[1]/nevts[1] #powheg+pythia8
scale2=lumi*xsecs[0]/nevts[0] #aMC@NLOFXFX+pythia8

fData=[]
path='../SingleMuonAll_2018UL.root'
files=glob.glob(path)
for f in files:
    fData.append(f)
print (len(fData))

fMC=[
    '../TTToSemileptonic_powheg_pythia8_2018UL.root',
    '../TTJets_amcatnloFXFX-pythia8_2018UL.root'
]

print (len(fMC))

1
2


In [21]:
canvases2 = []
for i in range(len(genBin_WSel)):
    c = ROOT.TCanvas("c%d"%i,"c%d"%i,10, 10, 750, 750 )
    canvases2.append(c)
    

for i in range(len(canvases2)):
    
    
    print (varNames_WSel[i], genBin_WSel[i], recoBin_WSel[i])
    gen_bin = genBin_WSel[i]
    reco_bin = recoBin_WSel[i]
    
    can=canvases2[i]
    
    MC1_reco=ROOT.TH1D("MC_reco Powheg+P8","MC_reco Powheg+P8;#tau_{1}^{(0.5)};Events/0.01",len(reco_bin)-1,reco_bin)
    MC1_gen=ROOT.TH1D("MC_gen Powheg+P8","MC_gen Powheg+P8;#tau_{1}^{(0.5)};Events/0.01",len(gen_bin)-1,gen_bin)
    Response1=ROOT.TH2D("Response matrix Powheg+P8","Response matrix Powheg+P8;#tau_{1}^{(0.5)} gen;#tau_{1}^{(0.5)} reco",len(gen_bin)-1,gen_bin,len(reco_bin)-1,reco_bin)
    MC2_reco=ROOT.TH1D("MC_reco aMC@NLO+Pythia8","MC_reco aMC@NLO+Pythia8;#tau_{1}^{(0.5)};Events/0.01",len(reco_bin)-1,reco_bin)
    MC2_gen=ROOT.TH1D("MC_gen aMC@NLO+Pythia8","MC_gen aMC@NLO+Pythia8;#tau_{1}^{(0.5)};Events/0.01",len(gen_bin)-1,gen_bin)
    Response2=ROOT.TH2D("Response matrix aMC@NLO+Pythia8","Response matrix aMC@NLO+Pythia8;#tau_{1}^{(0.5)} gen;#tau_{1}^{(0.5)} reco",len(gen_bin)-1,gen_bin,len(reco_bin)-1,reco_bin)
    
    print ("loading files")
    file_MC=np.zeros((len(fMC),),dtype=ROOT.TFile)
    file_MC[0]=ROOT.TFile.Open(fMC[0],'read')
    t=file_MC[0].Get('jetObservables')
    MC1_reco0 = t.Get(recoHist_WSel[i])
    #print (MC1_reco0.GetNbinsX())
    MC1_reco0.Scale(scale1)
    MC1_reco0.SetDirectory(0)
    MC1_gen0 = t.Get(accepgenHist_WSel[i])
    MC1_gen0.Scale(scale1)
    MC1_gen0.SetDirectory(0)
    Response1_0 = t.Get(response_WSel[i])
    Response1_0.Scale(scale1)
    Response1_0.SetDirectory(0)
    file_MC[0].Close()
    

    file_MC[1]=ROOT.TFile.Open(fMC[1],'read')
    t=file_MC[1].Get('jetObservables')
    MC2_reco0 = t.Get(recoHist_WSel[i])
    MC2_reco0.Scale(scale2)
    MC2_reco0.SetDirectory(0)
    MC2_gen0  = t.Get(accepgenHist_WSel[i])
    MC2_gen0.Scale(scale2)
    MC2_gen0.SetDirectory(0)
    Response2_0 = t.Get(response_WSel[i])
    Response2_0.Scale(scale2)
    Response2_0.SetDirectory(0)
    file_MC[1].Close()   
    
    
    
    print ("Re-filling histos")
    #print ("Filling resp")
    Fill1D(MC1_reco0,MC1_reco,reco_bin)
    Fill1D(MC1_gen0,MC1_gen,gen_bin)
    Fill2D(Response1_0,Response1,gen_bin,reco_bin)
    Fill1D(MC2_reco0,MC2_reco,reco_bin)
    Fill1D(MC2_gen0,MC2_gen,gen_bin)
    Fill2D(Response2_0,Response2,gen_bin,reco_bin)    #print ("Filling MC2 reco")
    Fill1D(MC2_reco0,MC2_reco,reco_bin)
    #print ("Filling MC2 gen")
    Fill1D(MC2_gen0,MC2_gen,gen_bin)
    
    can.cd()
    pad1 = ROOT.TPad("pad1", "Main",0,0.207,1.00,1.00,-1)
    pad2 = ROOT.TPad("pad2", "Ratio",0,0.00,1.00,0.30,-1)
    pad1.Draw()
    pad2.Draw()

    pad1.cd()

    if not(varNames_WSel[i] in ["tau_0p5_1", "tau_1_1", "tau32"]): legend1=ROOT.TLegend(0.65,0.65,0.90,0.88)
    else: legend1=ROOT.TLegend(0.15,0.65,0.35,0.88)
    legend1.SetFillStyle(0)
    legend1.SetBorderSize(0)

    print ("Unfolding and scaling histos")
    
    unf21,unf12=SelfClosure(Response1,MC2_reco,Response2,MC1_reco)
    Scale(MC1_gen)
    Scale(unf21)
    Scale(MC2_gen)
    Scale(unf12)
    
    unf21.SetLineColor(ROOT.kRed-7)
    unf21.SetMarkerColor(ROOT.kRed-7)
    unf21.SetMarkerStyle(26)
    unf21.SetMarkerSize(2)
    unf21.SetLineWidth(2)
    unf21.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d(%s)}'%HistLabels_WSel[i][1] )
    legend1.AddEntry(unf21,"aMC@NLO unfolded","lep")
    unf21.Draw("e0")

    MC2_gen.SetLineColor(ROOT.kBlue+7)
    MC2_gen.SetLineWidth(2)
    MC2_gen.SetMarkerSize(2)
    legend1.AddEntry(MC2_gen,"Gen aMC@NLO","l")
    MC2_gen.Draw("e0same")
    
    unf12.SetLineColor(ROOT.kGreen+3)
    unf12.SetMarkerColor(ROOT.kGreen+3)
    unf12.SetMarkerStyle(26)
    unf12.SetMarkerSize(2)
    unf12.SetLineWidth(2)
    legend1.AddEntry(unf12,"Powheg unfolded","lep")
    unf12.Draw("e0same")

    MC1_gen.SetLineColor(ROOT.kGray+2)
    MC1_gen.SetLineWidth(2)
    MC1_gen.SetMarkerSize(2)
    legend1.AddEntry(MC1_gen,"Gen Powheg","l")
    MC1_gen.Draw("e0same")
    
    legend1.Draw("same")
    can.Update()
    
    pad2.cd()
    ROOT.gStyle.SetOptFit(1)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.3)

    legend2=ROOT.TLegend(0.1,0.6,0.4,0.8)
    legend2.SetBorderSize(0)
    legend2.SetFillStyle(0)
    
    ratio1=unf21.Clone("ratio1")
    ratio1.Divide(MC2_gen)
    ratio1.SetTitle("")
    ratio1.GetXaxis().SetTitle(HistLabels_WSel[i][1])
    ratio1.SetTitleSize(0)
    ratio1.GetXaxis().SetTitleSize(0.12)
    ratio1.GetXaxis().SetLabelSize(0.12)
    ratio1.GetYaxis().SetTitle("(Unf.)/(Accep. gen)")
    ratio1.GetYaxis().SetTitleSize(0.11)
    ratio1.GetYaxis().SetTitleOffset(0.42)
    ratio1.GetYaxis().SetLabelSize(0.12)
    ratio1.GetYaxis().SetRangeUser(0.4,1.6)
    ratio1.SetMarkerStyle(22)
    ratio1.SetMarkerColor(ROOT.kGray+2)
    ratio1.SetLineColor(ROOT.kGray+2)
    ratio1.Draw("P0")
    legend2.AddEntry(ratio1,"Powheg+Pythia8","lep")
    
    ratio2=unf12.Clone("ratio1")
    ratio2.Divide(MC1_gen)
    ratio2.SetTitle("")
    ratio2.GetXaxis().SetTitle(HistLabels_WSel[i][1])
    ratio2.SetTitleSize(0)
    ratio2.GetXaxis().SetTitleSize(0.12)
    ratio2.GetXaxis().SetLabelSize(0.12)
    ratio2.GetYaxis().SetTitle("(Unf.)/(Accep. gen)")
    ratio2.GetYaxis().SetTitleSize(0.11)
    ratio2.GetYaxis().SetTitleOffset(0.42)
    ratio2.GetYaxis().SetLabelSize(0.12)
    ratio2.GetYaxis().SetRangeUser(0.4,1.6)
    ratio2.SetMarkerStyle(22)
    ratio2.SetMarkerColor(ROOT.kBlue+7)
    ratio2.SetLineColor(ROOT.kBlue+7)
    ratio2.Draw("P0 same")
    legend2.AddEntry(ratio2,"aMC@NLO+Pythia8","lep")
    
    legend2.Draw("Same")
    can.Update()
    can.Draw()
    can.Print("Plots/CrossClosureWSel_%s_UL18.png"%varNames_WSel[i] )
    MC2_gen.Delete()
    MC2_reco.Delete()
    Response1.Delete()
    MC1_gen.Delete()
    MC1_reco.Delete()
    Response2.Delete()
    can.Delete()

tau_0p5_1 array('d', [0.0, 0.2, 0.26, 0.32, 0.38, 0.44, 0.5, 0.56, 0.62, 0.7, 0.78, 0.9]) array('d', [0.0, 0.1, 0.2, 0.23, 0.26, 0.29, 0.32, 0.35, 0.38, 0.41, 0.44, 0.47, 0.5, 0.53, 0.56, 0.59, 0.62, 0.66, 0.7, 0.74, 0.78, 0.84, 0.9])
loading files
Re-filling histos
Unfolding and scaling histos
tau_1_1 array('d', [0.0, 0.1, 0.18, 0.24, 0.3, 0.36, 0.4, 0.46, 0.52, 0.6]) array('d', [0.0, 0.05, 0.1, 0.14, 0.18, 0.21, 0.24, 0.27, 0.3, 0.33, 0.36, 0.38, 0.4, 0.43, 0.46, 0.49, 0.52, 0.56, 0.6])
loading files
Re-filling histos
Unfolding and scaling histos
tau_2_1 array('d', [0.0, 0.04, 0.08, 0.12, 0.16, 0.2, 0.26, 0.32, 0.4, 0.56]) array('d', [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.23, 0.26, 0.29, 0.32, 0.36, 0.4, 0.48, 0.56])
loading files
Re-filling histos
Unfolding and scaling histos
tau_0p5_2 array('d', [0.0, 0.12, 0.18, 0.24, 0.3, 0.36, 0.42, 0.48, 0.54, 0.66]) array('d', [0.0, 0.06, 0.12, 0.15, 0.18, 0.21, 0.24, 0.27, 0.3, 0.33, 0.36, 0.39, 0.42, 0.45, 0.48, 0.

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c0
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c2
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c3
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c4
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c5
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c6
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c7
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c8
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c9
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c10
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c11
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c12
Info in <TUnfold::SetConstraint>: fConstraint=0
Info in <TUnfold::TUnfold>: underflow a